In [14]:
import pandas as pd, numpy as np, os, re

In [15]:
def parse_duration(x):
    if pd.isna(x): return 0.0
    s = str(x).strip()
    m = re.search(r'([0-9]+(?:\.[0-9]+)?)', s)
    if m:
        try:
            return float(m.group(1))
        except:
            return 0.0
    return 0.0

In [16]:
def run_pipeline(data_dir=".", out_dir="."):
    sms_to_seconds = 5.0

In [17]:

workspace_df = pd.read_csv("data/final_workspace_data.csv")
payment_df = pd.read_csv("data/final_payment_refund_data.csv")
integration_df = pd.read_csv("data/final_integration_data.csv")
calls_sms_df = pd.read_csv("data/final_calls_sms_detailed_data.csv")


In [18]:
workspace_df['member_emails'] = workspace_df['member_emails'].fillna('')
rows = []
for _, r in workspace_df.iterrows():
        owner = r['owner_email']
        rows.append({'workspace_id': r['workspace_id'], 'workspace_name': r['workspace_name'],
                     'owner_email': owner, 'user_email': owner, 'is_owner': True})
        if r['member_emails'].strip() != '':
            members = [m.strip() for m in r['member_emails'].split(',') if m.strip()!='']
            for m in members:
                rows.append({'workspace_id': r['workspace_id'], 'workspace_name': r['workspace_name'],
                             'owner_email': owner, 'user_email': m, 'is_owner': False})
workspace_users_df = pd.DataFrame(rows)


In [19]:

    # Payments/refunds -> net revenue
payment_df['time_dt'] = pd.to_datetime(payment_df['time'], unit='s')
payments = payment_df[payment_df['event']=='New Payment Made'].copy().rename(columns={'$email':'owner_email','Amount':'Amount_paid'})
payments_sum = payments.groupby('owner_email', as_index=False)['Amount_paid'].sum().rename(columns={'Amount_paid':'total_payments'})
first_payment = payments.groupby('owner_email', as_index=False)['time_dt'].min().rename(columns={'time_dt':'first_payment_time'})

refunds = payment_df[payment_df['event']=='Refund Granted'].copy().rename(columns={'User Email':'owner_email','Refund Amount':'Refund_Amount'})
refunds_sum = refunds.groupby('owner_email', as_index=False)['Refund_Amount'].sum().rename(columns={'Refund_Amount':'total_refunds'})
first_refund = refunds.groupby('owner_email', as_index=False)['time_dt'].min().rename(columns={'time_dt':'first_refund_time'})

owner_revenue = payments_sum.merge(refunds_sum, on='owner_email', how='left').merge(first_payment, on='owner_email', how='left').merge(first_refund, on='owner_email', how='left')
owner_revenue['total_refunds'] = owner_revenue['total_refunds'].fillna(0)
owner_revenue['net_revenue'] = owner_revenue['total_payments'] - owner_revenue['total_refunds']
owner_revenue = owner_revenue[owner_revenue['net_revenue'] > 0].copy()

In [20]:
# Integration windows
integration_df['event_time'] = pd.to_datetime(integration_df['time'], unit='s')
int_events = integration_df[integration_df['event'].isin(['Integration Connected','Integration Disconnected'])].sort_values(['distinct_id','Integration Platform','event_time'])
conn_windows = []
for (user, platform), grp in int_events.groupby(['distinct_id','Integration Platform']):
        starts = []
        for _, row in grp.iterrows():
            if row['event'] == 'Integration Connected':
                starts.append(row['event_time'])
            elif row['event'] == 'Integration Disconnected':
                if starts:
                    start = starts.pop(0)
                    conn_windows.append({'user_email': user, 'platform': platform, 'connected_from': start, 'connected_to': row['event_time']})
        for start in starts:
            conn_windows.append({'user_email': user, 'platform': platform, 'connected_from': start, 'connected_to': pd.NaT})
conn_windows_df = pd.DataFrame(conn_windows)
conn_windows_df = conn_windows_df.merge(workspace_users_df[['user_email','owner_email','workspace_id']], left_on='user_email', right_on='user_email', how='left')

In [22]:
 # Usage
calls_sms_df['event_time'] = pd.to_datetime(calls_sms_df['time'], unit='s')
calls_sms_df['call_seconds'] = calls_sms_df['Duration'].apply(parse_duration)
calls_sms_df['sms_segments'] = calls_sms_df['Segments'].fillna(0).astype(int)
calls_sms_df = calls_sms_df.rename(columns={'Owner':'owner_email','Email':'user_email','Platform':'platform','Workspace':'workspace_name'})
usage_df = calls_sms_df.merge(owner_revenue[['owner_email','first_payment_time','first_refund_time','net_revenue']], on='owner_email', how='left')
usage_df = usage_df[~usage_df['net_revenue'].isna()].copy()
cond_after = usage_df['event_time'] >= usage_df['first_payment_time']
cond_before = usage_df['first_refund_time'].isna() | (usage_df['event_time'] <= usage_df['first_refund_time'])
sage_df = usage_df[cond_after & cond_before].copy()

def has_active_connection(row):
        if row['platform'] in ('web','mobile'): return True
        user = row['user_email']; plat = row['platform']; et = row['event_time']
        wins = conn_windows_df[(conn_windows_df['user_email']==user) & (conn_windows_df['platform']==plat)]
        for _, w in wins.iterrows():
            start = w['connected_from']; end = w['connected_to']
            if pd.isna(end):
                if et >= start: return True
            else:
                if (et >= start) and (et <= end): return True
        return False

usage_df['valid_connection'] = usage_df.apply(has_active_connection, axis=1)
usage_df = usage_df[usage_df['valid_connection']].copy()
usage_df['usage_units'] = usage_df['call_seconds'] + usage_df['sms_segments'] 
usage_owner_platform = usage_df.groupby(['owner_email','platform'], as_index=False)['usage_units'].sum()
owner_total_usage = usage_owner_platform.groupby('owner_email', as_index=False)['usage_units'].sum().rename(columns={'usage_units':'total_usage_units'})
usage_owner_platform = usage_owner_platform.merge(owner_revenue[['owner_email','net_revenue']], on='owner_email', how='left').merge(owner_total_usage, on='owner_email', how='left')
usage_owner_platform['attributed_revenue'] = usage_owner_platform.apply(lambda r: (r['usage_units']/r['total_usage_units'])*r['net_revenue'] if r['total_usage_units']>0 else 0.0, axis=1)
usage_attrib_platform = usage_owner_platform.groupby('platform', as_index=False)['attributed_revenue'].sum().rename(columns={'attributed_revenue':'revenue'})
owners_with_no_usage = owner_revenue[~owner_revenue['owner_email'].isin(owner_total_usage['owner_email'])]
no_usage_sum = owners_with_no_usage['net_revenue'].sum()
if no_usage_sum>0:
        usage_attrib_platform = usage_attrib_platform.append({'platform':'No Usage','revenue':no_usage_sum}, ignore_index=True)
usage_attrib_platform['revenue'] = usage_attrib_platform['revenue'].round(2)


In [23]:
# First-touch
first_touch_rows = []
for _, orow in owner_revenue.iterrows():
        owner = orow['owner_email']; fp = orow['first_payment_time']
        cand = conn_windows_df[(conn_windows_df['owner_email']==owner) & (conn_windows_df['connected_from']>=fp)].copy()
        if cand.empty:
            first_touch_rows.append({'owner_email':owner, 'platform':'No Integration', 'net_revenue':orow['net_revenue']})
        else:
            cand = cand.sort_values('connected_from')
            plat = cand.iloc[0]['platform']
            first_touch_rows.append({'owner_email':owner, 'platform':plat, 'net_revenue':orow['net_revenue']})
first_touch_df = pd.DataFrame(first_touch_rows)
first_touch_platform = first_touch_df.groupby('platform', as_index=False)['net_revenue'].sum().rename(columns={'net_revenue':'revenue'})
first_touch_platform['revenue'] = first_touch_platform['revenue'].round(2)

In [24]:
# Position-based
position_rows = []
for _, orow in owner_revenue.iterrows():
        owner = orow['owner_email']; fp = orow['first_payment_time']
        cand = conn_windows_df[(conn_windows_df['owner_email']==owner) & (conn_windows_df['connected_from']>=fp)].copy()
        if cand.empty:
            position_rows.append({'owner_email':owner, 'platform':'No Integration', 'weight':1.0, 'net_revenue':orow['net_revenue']})
            continue
        cand = cand.sort_values('connected_from')
        platforms = list(dict.fromkeys(cand['platform'].tolist()))
        total = len(platforms)
        for idx, plat in enumerate(platforms, start=1):
            if total == 1:
                w = 1.0
            elif total == 2:
                w = 0.6 if idx==1 else 0.4
            else:
                if idx==1 or idx==total:
                    w = 0.4
                else:
                    w = 0.2 / (total - 2)
            position_rows.append({'owner_email':owner, 'platform':plat, 'weight':w, 'net_revenue':orow['net_revenue']})
position_df = pd.DataFrame(position_rows)
position_df['attributed_revenue'] = position_df['weight'] * position_df['net_revenue']
position_platform = position_df.groupby('platform', as_index=False)['attributed_revenue'].sum().rename(columns={'attributed_revenue':'revenue'})
position_platform['revenue'] = position_platform['revenue'].round(2)

In [26]:
first_touch_platform.to_csv(os.path.join("output/", "first_touch_attribution.csv"), index=False)
usage_attrib_platform.to_csv(os.path.join("output/", "usage_based_attribution.csv"), index=False)
position_platform.to_csv(os.path.join("output/", "chosen_attribution.csv"), index=False)